# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [15]:
#Your code here
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

df = pd.read_csv("homepage_actions.csv")

# basic structure
print("rows, cols:", df.shape)
print("columns:", df.columns.tolist())

# how many unique users
print("unique users (id):", df["id"].nunique())

# how many users viewed vs clicked
print("\naction counts:\n", df["action"].value_counts())

# investigate id-level behavior
user_actions = df.groupby("id")["action"].apply(set)

# how many viewers also clicked
view_and_click = user_actions.apply(lambda s: ("view" in s) and ("click" in s)).sum()
print("\nusers who viewed and clicked:", view_and_click)

# check anomalies: clicks without views
clicked_no_view = user_actions.apply(lambda s: ("click" in s) and ("view" not in s)).sum()
print("users who clicked without viewing:", clicked_no_view)

# check overlap between control and experiment groups
groups_per_user = df.groupby("id")["group"].nunique()
overlap = (groups_per_user > 1).sum()
print("users appearing in more than one group:", overlap)


rows, cols: (8188, 4)
columns: ['timestamp', 'id', 'group', 'action']
unique users (id): 6328

action counts:
 view     6328
click    1860
Name: action, dtype: int64

users who viewed and clicked: 1860
users who clicked without viewing: 0
users appearing in more than one group: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [17]:
#Your code here
# user-level dataset
user_level = df.groupby(["id","group"])["action"].apply(
    lambda x: (x == "click").any()
).reset_index(name="clicked")

# clicks and users per group
group_stats = user_level.groupby("group")["clicked"].agg(
    total_clicks="sum",
    total_users="count"
)

# click-through rates
control_ctr = group_stats.loc["control","total_clicks"] / group_stats.loc["control","total_users"]
experiment_ctr = group_stats.loc["experiment","total_clicks"] / group_stats.loc["experiment","total_users"]

# pooled proportion
pooled_ctr = group_stats["total_clicks"].sum() / group_stats["total_users"].sum()

# z-score
standard_error = math.sqrt(
    pooled_ctr * (1 - pooled_ctr) *
    (1/group_stats.loc["control","total_users"] +
     1/group_stats.loc["experiment","total_users"])
)

z_score = (experiment_ctr - control_ctr) / standard_error

# one-sided p-value (experiment > control)
p_value = 1 - norm.cdf(z_score)

print("control CTR:", control_ctr)
print("experiment CTR:", experiment_ctr)
print("z-score:", z_score)
print("p-value:", p_value)

"""
Since the p-value is below 0.05, we reject the null hypothesis and conclude that the experimental homepage performs better than the control homepage.
"""


control CTR: 0.2797118847539016
experiment CTR: 0.3097463284379172
z-score: 2.618563885349469
p-value: 0.004415037788297882


'\nSince the p-value is below 0.05, we reject the null hypothesis and conclude that the experimental homepage performs better than the control homepage.\n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [18]:
#Your code here
# expected clicks in experiment if it had control CTR
expected_clicks_experiment = experiment_ctr * 0  # placeholder
expected_clicks_experiment = (
    control_ctr * group_stats.loc["experiment", "total_users"]
)

expected_clicks_experiment


838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [19]:
#Your code here
# actual clicks in experiment
actual_clicks_experiment = group_stats.loc["experiment", "total_clicks"]

# standard deviation using binomial variance
std_dev = math.sqrt(
    group_stats.loc["experiment", "total_users"] *
    control_ctr *
    (1 - control_ctr)
)

# number of standard deviations from expected value (z-score)
z_verification = (
    actual_clicks_experiment - expected_clicks_experiment
) / std_dev

z_verification


3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here
from scipy.stats import norm

# one-sided p-value (experiment > control)
p_value_verification = 1 - norm.cdf(z_verification)

p_value_verification
"""
The p-value is less than 0.05, we reject the null hypothesis and conclude that the experimental homepage performs better than the control homepage.
"""

'\nIf the p-value is less than 0.05, we reject the null hypothesis and conclude that the experimental homepage performs better than the control homepage.\n'

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
Yes. The verification approach produces a z-score and p-value that are very close to those from the earlier two-proportion z-test. Although the calculations differ slightly because this method treats the control click-through rate as fixed, both approaches lead to the same conclusion that the experimental homepage performs significantly better than the control homepage.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.